In [10]:
import sys
from pathlib import Path

import pandas as pd

# point to project root
project_root = Path("/Users/dpro/projects/food_desert")

# add src to import path if you want paths, etc.
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from food_desert import paths  # noqa: F401

euclid_path = project_root / "data" / "interim" / "parcel_nearest_grocer_mask.csv"
path_path = project_root / "data" / "interim" / "parcel_nearest_grocer_path_mask.csv"
parcels_raw_path = project_root / "data" / "raw" / "Assessment_Parcels_20251112.csv"


In [11]:
euclid_df = pd.read_csv(euclid_path)
path_df = pd.read_csv(path_path)

# sanity check columns
euclid_df.columns, path_df.columns


(Index(['Roll Number', 'neighbourhood_id', 'name', 'population', 'residents',
        'dist_to_grocer_m', 'grocer_chain', 'grocer_store'],
       dtype='object'),
 Index(['Roll Number', 'neighbourhood_id', 'name', 'population', 'residents',
        'dist_to_grocer_path_m'],
       dtype='object'))

In [12]:
# merge on Roll Number + neighbourhood_id + name just to be strict
merged = euclid_df.merge(
    path_df,
    on=["Roll Number", "neighbourhood_id", "name"],
    how="inner",
    suffixes=("_euclid", "_path"),
)

merged.shape


(219717, 11)

In [13]:
# rename distance columns if needed
# adjust these names if your euclidean script used something else
dist_e = "dist_to_grocer_m"
dist_p = "dist_to_grocer_path_m"

if dist_e not in merged.columns:
    raise ValueError(f"missing {dist_e} in merged")
if dist_p not in merged.columns:
    raise ValueError(f"missing {dist_p} in merged")

merged["ratio_path_to_euclid"] = merged[dist_p] / merged[dist_e]

merged[[dist_e, dist_p, "ratio_path_to_euclid"]].describe()


,dist_to_grocer_m,dist_to_grocer_path_m,ratio_path_to_euclid
count,219717.000000,219458.000000,219458.000000
mean,936.311952,1294.618692,1.420375
std,515.303872,753.468858,0.557303
min,23.708627,0.000000,0.000000
25%,550.739748,761.446669,1.160640
50%,859.247751,1175.982967,1.325341
75%,1238.711197,1703.840398,1.549194
max,5283.168935,9355.844996,15.203502


In [14]:
merged["ratio_path_to_euclid"].quantile([0.5, 0.9, 0.95, 0.99])


0.50    1.325341
0.90    1.915294
0.95    2.248395
0.99    3.520819
Name: ratio_path_to_euclid, dtype: float64

In [15]:
# largest ratios first
worst = (
    merged[["Roll Number", "neighbourhood_id", "name",
            dist_e, dist_p, "ratio_path_to_euclid"]]
    .sort_values("ratio_path_to_euclid", ascending=False)
)

worst.head(20)


,Roll Number,neighbourhood_id,name,dist_to_grocer_m,dist_to_grocer_path_m,ratio_path_to_euclid
128474,8008094100,1644,Dakota Crossing,87.250255,1326.509381,15.203502
67258,5009111600,634,Leila-McPhillips Triangle,48.779298,722.413661,14.809841
128475,8008094300,1644,Dakota Crossing,95.605363,1326.509381,13.874843
67259,5009111800,634,Leila-McPhillips Triangle,57.610607,722.413661,12.539595
128476,8008094500,1644,Dakota Crossing,102.837679,1265.170928,12.302601
205738,14061405000,702,Luxton,33.861189,411.043031,12.139061
132717,8081217565,1670,St. Vital Perimeter South,583.297660,6798.795145,11.655790
67257,5009111400,634,Leila-McPhillips Triangle,63.680284,722.413661,11.344385
128473,8008093900,1644,Dakota Crossing,125.411301,1326.509381,10.577272
128472,8008093700,1644,Dakota Crossing,131.717529,1326.509381,10.070864


In [16]:
parcels_raw = pd.read_csv(parcels_raw_path, low_memory=False)

# pull just a few human-readable fields
addr_cols = [
    "Roll Number",
    "Full Address",
    "Street Number",
    "Street Name",
    "Street Type",
    "Street Direction",
]

addr_cols = [c for c in addr_cols if c in parcels_raw.columns]
parcels_addr = parcels_raw[addr_cols].copy()

audit = merged.merge(parcels_addr, on="Roll Number", how="left")

audit.head()


,Roll Number,neighbourhood_id,name,population_euclid,residents_euclid,dist_to_grocer_m,grocer_chain,grocer_store,population_path,residents_path,dist_to_grocer_path_m,ratio_path_to_euclid,Full Address,Street Number,Street Name,Street Type,Street Direction
0,1000001000,93,Wilkes South,815,2,1779.519861,Red-River Co-op,Seasons,815,2,3362.534582,1.889574,1636 MCCREARY ROAD,"1,636",MCCREARY,ROAD,NaN
1,1000005500,93,Wilkes South,815,6,1634.613819,Red-River Co-op,Seasons,815,6,3362.534582,2.057082,1584 MCCREARY ROAD,"1,584",MCCREARY,ROAD,NaN
2,1000008000,93,Wilkes South,815,1,1590.978720,Red-River Co-op,Seasons,815,1,3362.534582,2.113501,1574 MCCREARY ROAD,"1,574",MCCREARY,ROAD,NaN
3,1000008200,93,Wilkes South,815,4,1562.186218,Red-River Co-op,Seasons,815,4,3362.534582,2.152454,1550 MCCREARY ROAD,"1,550",MCCREARY,ROAD,NaN
4,1000008400,93,Wilkes South,815,2,1519.454985,Red-River Co-op,Seasons,815,2,3362.534582,2.212987,1538 MCCREARY ROAD,"1,538",MCCREARY,ROAD,NaN


In [ ]:
def inspect_roll(roll_number: str | int) -> pd.DataFrame:
    """Show distances and address for one parcel (by Roll Number)."""
    mask = audit["Roll Number"] == roll_number
    rows = audit.loc[mask, [
        "Roll Number",
        "Full Address" if "Full Address" in audit.columns else "Street Name",
        "neighbourhood_id",
        "name",
        dist_e,
        dist_p,
        "ratio_path_to_euclid",
    ]]
    if rows.empty:
        print(f"no rows found for Roll Number={roll_number}")
    return rows


# example: plug in one you checked against Google
inspect_roll("3092975610")  # replace with a real roll number


no rows found for Roll Number=03092975610


,Roll Number,Full Address,neighbourhood_id,name,dist_to_grocer_m,dist_to_grocer_path_m,ratio_path_to_euclid


In [18]:
# e.g. flag parcels where path is more than 2x euclid
suspect = audit[audit["ratio_path_to_euclid"] > 2.0].copy()

len(suspect), suspect.head(10)


(18055,
     Roll Number  neighbourhood_id          name  population_euclid  \
 1    1000005500                93  Wilkes South                815   
 2    1000008000                93  Wilkes South                815   
 3    1000008200                93  Wilkes South                815   
 4    1000008400                93  Wilkes South                815   
 5    1000008500                93  Wilkes South                815   
 6    1000013200                93  Wilkes South                815   
 7    1000013300                93  Wilkes South                815   
 8    1000013600                93  Wilkes South                815   
 9    1000013700                93  Wilkes South                815   
 10   1000014000                93  Wilkes South                815   
 
     residents_euclid  dist_to_grocer_m     grocer_chain grocer_store  \
 1                  6       1634.613819  Red-River Co-op      Seasons   
 2                  1       1590.978720  Red-River Co-op      S